# **ReMASTER: Extension of Market-Guided Stock Transformer for Stock Price Forecasting Using Novel Stock Indices**

**Developed By:** Anonymous

Some tasks in this assignment can take a long time if you run it on the CPU. For example, based on our solution of Exercise 3 Task 4 (Transfer Learning: finetuning of a pretrained model (resnet18)), it will take roughly 2 hours to train the model end-to-end (complete model and not only the last fc layer) for 1 epoch on CPU. Hence, we highly recommend you try to train your model on GPU.

To do so, first you need to enable GPU on Colab (this will restart the runtime). Click Runtime-> Change runtime type and select the Hardware accelerator there. You can then run the following code to see if the GPU is correctly initialized and available.

Note: If you would like to avoid GPU overages on Colab, we would suggest writing and debugging your code before switching on the GPU runtime. Otherwise, the time you spent debugging code will likely count against your GPU usage. Once you have the code running, you can switch on the GPU runtime and train the model much faster.

In [23]:
import torch
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Replace "0" with the GPU ID(s) you want to use

print(f'Can I use GPU now? -- {torch.cuda.is_available()}')
# if torch.cuda.is_available():
#     # device = torch.device("cuda:0")  # Use the first CUDA device
#     device = torch.device("cuda")
#     print("Using CUDA device:", device)

Can I use GPU now? -- True


If you wish to avoid Colab's GPU overages, consider writing and debugging code on the CPU first before switching to the GPU runtime.



---



### **Step 1: Install Required Libraries**
Run the following commands to install necessary libraries like pandas, torch, and qlib (from GitHub):

In [24]:
# # install requirements
# # install qlib library to load in datasets
# # Install pandas and torch with specific versions
# # !pip install pandas==1.5.3 torch==1.11.0


# !pip freeze > requirements.txt
# !pip install pandas==1.5.3
# !pip install torch==1.11.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html


# # Install dependencies for qlib


# !pip install --upgrade cython setuptools

# # Install qlib from GitHub, as the PyPI version only supports Python 3.7 and 3.8


# !pip install git+https://github.com/microsoft/qlib.git

# # Install any remaining dependencies
# # !pip install pyqlib
# # !pip install pylib==0.9.1.99




### **Step 2: Import Libraries**


In [25]:
import time
import os
import torch
import copy
import numpy as np
import pandas as pd
import pickle
import qlib
from torch.utils.data import DataLoader, Sampler
import torch.optim as optim

### **Step 3: Verify GPU Availability and Set Up Data**
Initialize qlib and check GPU status:




In [26]:
qlib.init()
print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')

[1791:MainThread](2024-11-12 06:49:05,135) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
INFO:qlib.Initialization:default_conf: client.
[1791:MainThread](2024-11-12 06:49:05,137) WARNING - qlib.Initialization - [__init__.py:64] - auto_path is False, please make sure None is mounted
[1791:MainThread](2024-11-12 06:49:05,140) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
INFO:qlib.Initialization:qlib successfully initialized based on client settings.
[1791:MainThread](2024-11-12 06:49:05,142) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/root/.qlib/qlib_data/cn_data')}
INFO:qlib.Initialization:data_path={'__DEFAULT_FREQ': PosixPath('/root/.qlib/qlib_data/cn_data')}


Can I can use GPU now? -- True


### **Step 4: Load Data from Google Drive**
Set up the data. Load in from google drive mount.

In [27]:
from google.colab import drive
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Set the path to your data folder
data_path = '/content/drive/My Drive/ECE 570/data'

# Step 3: List files in the folder (to verify)
print("Files in data folder:")
for filename in os.listdir(data_path):
    print(filename)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files in data folder:
csi_market_information.csv
csi300.zip
NQ_5Years_8_11_2024.csv
old_nasdaq
csi_model
csi300
csi800
NQ100


## **BASE_MODEL.PY**
### **Helper Functions and Classes**
Define the necessary functions and classes for data processing and model setup.

In [28]:
from torch.utils.data import DataLoader
from torch.utils.data import Sampler
import torch
import torch.optim as optim

def calc_ic(pred, label):
    # Flatten pred and label to ensure they’re 1-dimensional
    pred = pred.reshape(-1)
    label = label.reshape(-1)

    # print(f"calc_ic - Pred shape after reshape: {pred.shape}, Label shape after reshape: {label.shape}")

    # Create DataFrame and calculate IC and RIC
    df = pd.DataFrame({'pred': pred, 'label': label})
    ic = df['pred'].corr(df['label'])
    ric = df['pred'].corr(df['label'], method='spearman')

    # print(f"calc_ic - IC: {ic}, RIC: {ric}")
    return ic, ric

**Custom Data Sampler**

Key changes for ReMASTER:

*   DailyBatchSamplerRandom: use .index.get_level_values('datetime') to group by datetime for pandas df structure.
*   Make all shuffle false
*   More changes commented below






In [29]:
from torch.utils.data import DataLoader
from torch.utils.data import Sampler
import torch
import torch.optim as optim


# MASTER'S IMPLEMENTATION (untouched)

# class DailyBatchSamplerRandom(Sampler):
#     def __init__(self, data_source, shuffle=False):
#         self.data_source = data_source
#         self.shuffle = shuffle
#         # calculate number of samples in each batch
#         self.daily_count = pd.Series(index=self.data_source.get_index()).groupby("datetime").size().values
#         self.daily_index = np.roll(np.cumsum(self.daily_count), 1)  # calculate begin index of each batch
#         self.daily_index[0] = 0

#     def __iter__(self):
#         if self.shuffle:
#             index = np.arange(len(self.daily_count))
#             np.random.shuffle(index)
#             for i in index:
#                 yield np.arange(self.daily_index[i], self.daily_index[i] + self.daily_count[i])
#         else:
#             for idx, count in zip(self.daily_index, self.daily_count):
#                 yield np.arange(idx, idx + count)

#     def __len__(self):
#         return len(self.data_source)

# ====================================================================================================================================================================================
#
#                           below is ReMASTER
#
# ====================================================================================================================================================================================

# because nq100 is alr pandas df. added helpers as well
# import torch
# from torch.utils.data import Dataset, DataLoader, Sampler
# import pandas as pd
# import numpy as np
# from torch.utils.data import TensorDataset, DataLoader

# adjust index error
class DailyBatchSamplerRandom(Sampler):
    def __init__(self, data_source, shuffle=False):
        self.data_source = data_source
        self.shuffle = shuffle
        # calculate number of samples in each batch
        self.daily_count = pd.Series(index=self.data_source.index.get_level_values('datetime')).groupby("datetime").size().values
        self.daily_index = np.roll(np.cumsum(self.daily_count), 1)  # calculate begin index of each batch
        self.daily_index[0] = 0

    def __iter__(self):
        if self.shuffle:
            index = np.arange(len(self.daily_count))
            np.random.shuffle(index)
            for i in index:
                yield np.arange(self.daily_index[i], self.daily_index[i] + self.daily_count[i])
        else:
            for idx, count in zip(self.daily_index, self.daily_count):
                yield np.arange(idx, idx + count)

    def __len__(self):
        return len(self.data_source)



---



# **MASTER.PY**
### **Model Definition**
Define the architecture for ReMASTER model with position encoding, self-attention, and gated layers.

In [30]:
# import torch.nn as nn
# from torch.nn.modules.linear import Linear
# from torch.nn.modules.dropout import Dropout
# from torch.nn.modules.normalization import LayerNorm
# import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, LayerNorm, Dropout
import math
import copy
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Sampler


# master



class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # reMASTER adds this
        # Initialize with max expected size, or dynamically expand based on the input shape
        if x.shape[1] > self.pe.size(0):
            self.pe = torch.nn.functional.pad(self.pe, (0, 0, 0, x.shape[1] - self.pe.size(0)))

        return x + self.pe[:x.shape[1], :]

**Implementation of SAttention, TAttention, Gate, TemporalAttention, and MASTER classes**

In [31]:
# -------------------------------------for ReMASTER import TensorDataset and pandas------------------------------
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import datetime
import numpy as np
import torch
import torch.nn.functional as F

class SAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout):
        super().__init__()

        self.d_model = d_model
        self.nhead = nhead
        self.temperature = math.sqrt(self.d_model/nhead)

        self.qtrans = nn.Linear(d_model, d_model, bias=False)
        self.ktrans = nn.Linear(d_model, d_model, bias=False)
        self.vtrans = nn.Linear(d_model, d_model, bias=False)

        attn_dropout_layer = []
        for i in range(nhead):
            attn_dropout_layer.append(Dropout(p=dropout))
        self.attn_dropout = nn.ModuleList(attn_dropout_layer)

        self.norm1 = LayerNorm(d_model, eps=1e-5)
        self.norm2 = LayerNorm(d_model, eps=1e-5)
        self.ffn = nn.Sequential(
            Linear(d_model, d_model),
            nn.ReLU(),
            Dropout(p=dropout),
            Linear(d_model, d_model),
            Dropout(p=dropout)
        )

    def forward(self, x):
        x = self.norm1(x)
        q = self.qtrans(x)  # Shape: [batch_size, seq_len, d_model]
        k = self.ktrans(x)  # Shape: [batch_size, seq_len, d_model]
        v = self.vtrans(x)  # Shape: [batch_size, seq_len, d_model]

        # print(f'q shape after qtrans: {q.shape}')  # Debugging
        # print(f'k shape after ktrans: {k.shape}')  # Debugging
        # print(f'v shape after vtrans: {v.shape}')  # Debugging

        dim = int(self.d_model / self.nhead)
        att_output = []
        for i in range(self.nhead):
            if i == self.nhead - 1:
                qh = q[:, :, i * dim:]
                kh = k[:, :, i * dim:]
                vh = v[:, :, i * dim:]
            else:
                qh = q[:, :, i * dim:(i + 1) * dim]
                kh = k[:, :, i * dim:(i + 1) * dim]
                vh = v[:, :, i * dim:(i + 1) * dim]

            # print(f'qh shape (head {i}): {qh.shape}')  # Debugging
            # print(f'kh shape (head {i}): {kh.shape}')  # Debugging

            atten_ave_matrixh = torch.softmax(torch.matmul(qh, kh.transpose(1, 2)) / self.temperature, dim=-1)
            if self.attn_dropout:
                atten_ave_matrixh = self.attn_dropout[i](atten_ave_matrixh)
            att_output.append(torch.matmul(atten_ave_matrixh, vh))

        att_output = torch.concat(att_output, dim=-1)

        # FFN
        xt = x + att_output
        xt = self.norm2(xt)
        att_output = xt + self.ffn(xt)

        return att_output

class TAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout):
        super().__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.qtrans = nn.Linear(d_model, d_model, bias=False)
        self.ktrans = nn.Linear(d_model, d_model, bias=False)
        self.vtrans = nn.Linear(d_model, d_model, bias=False)

        self.attn_dropout = []
        if dropout > 0:
            for i in range(nhead):
                self.attn_dropout.append(Dropout(p=dropout))
            self.attn_dropout = nn.ModuleList(self.attn_dropout)

        # input LayerNorm
        self.norm1 = LayerNorm(d_model, eps=1e-5)
        # FFN layerNorm
        self.norm2 = LayerNorm(d_model, eps=1e-5)
        # FFN
        self.ffn = nn.Sequential(
            Linear(d_model, d_model),
            nn.ReLU(),
            Dropout(p=dropout),
            Linear(d_model, d_model),
            Dropout(p=dropout)
        )

    def forward(self, x):
        x = self.norm1(x)
        q = self.qtrans(x)
        k = self.ktrans(x)
        v = self.vtrans(x)

        dim = int(self.d_model / self.nhead)
        att_output = []
        # ====================================================================================
        #
        # REMASTER FOR DIMENSION FIX
        #
        # ====================================================================================
        for i in range(self.nhead):
            if q.dim() == 3:  # Standard case, 3D tensor
                qh = q[:, :, i * dim:(i + 1) * dim]
                kh = k[:, :, i * dim:(i + 1) * dim]
                vh = v[:, :, i * dim:(i + 1) * dim]
            elif q.dim() == 2:  # Edge case, 2D tensor
                qh = q[:, i * dim:(i + 1) * dim]
                kh = k[:, i * dim:(i + 1) * dim]
                vh = v[:, i * dim:(i + 1) * dim]
            # Ensure `qh` and `kh` have at least 3 dimensions by unsqueezing if needed
            if qh.dim() == 2:
                qh = qh.unsqueeze(0)
            if kh.dim() == 2:
                kh = kh.unsqueeze(0)

                # master implementation
                # below

        # for i in range(self.nhead):
        #     if i==self.nhead-1:
        #         qh = q[:, :, i * dim:]
        #         kh = k[:, :, i * dim:]
        #         vh = v[:, :, i * dim:]
        #     else:
        #         qh = q[:, :, i * dim:(i + 1) * dim]
        #         kh = k[:, :, i * dim:(i + 1) * dim]
        #         vh = v[:, :, i * dim:(i + 1) * dim]
            try:
                atten_ave_matrixh = torch.softmax(torch.matmul(qh, kh.transpose(1, 2)), dim=-1)
            except IndexError as e:
                print(f"Error: {e}")
                print(f"qh shape: {qh.shape}, kh shape: {kh.shape}")
            # atten_ave_matrixh = torch.softmax(torch.matmul(qh, kh.transpose(1, 2)), dim=-1)
            if self.attn_dropout:
                atten_ave_matrixh = self.attn_dropout[i](atten_ave_matrixh)
            att_output.append(torch.matmul(atten_ave_matrixh, vh))
        att_output = torch.concat(att_output, dim=-1)

        # FFN
        xt = x + att_output
        xt = self.norm2(xt)
        att_output = xt + self.ffn(xt)

        return att_output

class Gate(nn.Module):
    def __init__(self, d_input, d_output,  beta=1.0):
        super().__init__()
        self.trans = nn.Linear(d_input, d_output)
        self.d_output =d_output
        self.t = beta

    def forward(self, gate_input):
        output = self.trans(gate_input)
        output = torch.softmax(output/self.t, dim=-1)
        return self.d_output*output


class TemporalAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.trans = nn.Linear(d_model, d_model, bias=False)

    def forward(self, z):
        h = self.trans(z) # [N, T, D]
        query = h[:, -1, :].unsqueeze(-1)
        lam = torch.matmul(h, query).squeeze(-1)  # [N, T, D] --> [N, T]
        lam = torch.softmax(lam, dim=1).unsqueeze(1)
        output = torch.matmul(lam, z).squeeze(1)  # [N, 1, T], [N, T, D] --> [N, 1, D]
        return output

class MASTER(nn.Module):
  # ReMASTER: change feat to 22, start to 0, end to 22
    def __init__(self, d_feat=22, d_model=256, t_nhead=4, s_nhead=2, T_dropout_rate=0.5, S_dropout_rate=0.5,
                 gate_input_start_index=0, gate_input_end_index=22, beta=None):
        super(MASTER, self).__init__()
        # market
        self.gate_input_start_index = gate_input_start_index
        self.gate_input_end_index = gate_input_end_index
        self.d_gate_input = (gate_input_end_index - gate_input_start_index) # F'
        self.feature_gate = Gate(self.d_gate_input, d_feat, beta=beta)

        self.layers = nn.Sequential(
            # feature layer
            nn.Linear(d_feat, d_model),
            PositionalEncoding(d_model),
            # intra-stock aggregation
            TAttention(d_model=d_model, nhead=t_nhead, dropout=T_dropout_rate),
            # inter-stock aggregation
            SAttention(d_model=d_model, nhead=s_nhead, dropout=S_dropout_rate),
            TemporalAttention(d_model=d_model),
            # decoder
            nn.Linear(d_model, 1)
        )

    def forward(self, x):
        src, gate_input = x, x  # Assuming src and gate_input start from x

        # print(f"Initial src shape: {src.shape}, Initial gate_input shape: {gate_input.shape}")

        # Apply feature gate transformation on gate_input and add an extra dimension
        # Pad gate_input to have dimensions close to src, accounting for the unsqueeze
        gate_dims = len(gate_input.shape)
        src_dims = len(src.shape)

        if gate_dims < src_dims - 2:
          gate_input = self.feature_gate(gate_input).unsqueeze(1)
          # print(f"gate_input after feature_gate transformation: {gate_input.shape}")

        # Check if src and gate_input are already aligned after the transformation
        if src.shape == gate_input.shape:
            return src * gate_input

        if gate_dims < src_dims - 2:
            # Pad gate_input to match src dimensions minus the unsqueeze dimension
            gate_padding = [
                (0, src.size(i) - gate_input.size(i)) if i < gate_dims and gate_input.size(i) < src.size(i) else (0, 0)
                for i in range(src_dims)
            ]
            gate_padding = sum(gate_padding, ())  # Flatten for F.pad
            gate_input = F.pad(gate_input, gate_padding, "constant", 0)
            # print(f"Padded gate_input shape: {gate_input.shape}")

        # Final alignment check
        assert src.shape == gate_input.shape, f"Shapes still not aligned: src {src.shape}, gate_input {gate_input.shape}"

        # Element-wise multiplication
        return src * gate_input


# --------------------------------------------------------------------------------------------------------------
# class sequencemodel
# ORIGINALLY PART OF BASE_MODEL.PY
#   ** moved here to avoid import since this doesn't work in colab notebooks
# --------------------------------------------------------------------------------------------------------------
class SequenceModel():
  # changed save path to fit with colab dir
    def __init__(self, n_epochs, lr, GPU=None, seed=None, train_stop_loss_thred=None, save_path = '/content/drive/My Drive/ECE 570/model/', save_prefix= ''):
        self.n_epochs = n_epochs
        self.lr = lr
        self.device = torch.device(f"cuda:{GPU}" if torch.cuda.is_available() else "cpu")
        self.seed = seed
        self.train_stop_loss_thred = train_stop_loss_thred

        if self.seed is not None:
            np.random.seed(self.seed)
            torch.manual_seed(self.seed)
        self.fitted = False

        self.model = None
        self.train_optimizer = None

        self.save_path = save_path
        self.save_prefix = save_prefix

        # FOR FORWARD FUNCTION DEBUGGING IN REMASTER
        self._is_training = False  # Private flag

    def set_training_mode(self, training=True):
        """Set the model to training or evaluation mode"""
        self._is_training = training
        self.model.train(training)

    def is_training(self):
        """Returns True if model is in training mode, otherwise False."""
        return self._is_training


    def init_model(self):
        if self.model is None:
            raise ValueError("model has not been initialized")

        self.train_optimizer = optim.Adam(self.model.parameters(), self.lr)
        self.model.to(self.device)

    # def loss_fn(self, pred, label):
    #     mask = ~torch.isnan(label)
    #     loss = (pred[mask]-label[mask])**2
    #     return torch.mean(loss)

    # ==============================================================================
    #
    # REMASTER : ensure that tensor indicies dont overflow
    #
    # ==============================================================================

    def loss_fn(self, pred, label):
        # Check if `pred` and `label` have matching shapes
        # print(f"\nPrediction shape: {pred.shape}, Label shape: {label.shape}")
        if pred.shape != label.shape:
            # If `label` has an additional dimension (e.g., [204, 1] vs. [204]),
            # then `squeeze` it down to match `pred` if `pred` is [204, 22] or similar.
            if len(label.shape) < len(pred.shape):
                label = label.unsqueeze(-1).expand_as(pred)
            elif len(label.shape) > len(pred.shape):
                label = label.squeeze(-1)
            else:
                # Reshape pred if they have the same number of dimensions but different shapes
                pred = pred.expand_as(label)
        # print(f"Adjusted Prediction shape: {pred.shape}, Adjusted Label shape: {label.shape}")

        # Create mask for non-NaN values
        mask = ~torch.isnan(label)

        # Calculate loss (use the mask to ignore NaNs)
        loss = (pred[mask] - label[mask]) ** 2

        # ========================================================================================
        #
        # REMASTER to safeguard
        #
        # ========================================================================================
        if mask.sum() == 0:  # Safeguard against no valid data points
          return torch.tensor(0.0, device=label.device)
        return torch.mean(loss)

    def train_epoch(self, data_loader):
        print("========================================================================================================")
        print("Training...")
        print("========================================================================================================")
        self.set_training_mode(True)  # Training mode
        self.model.train()
        losses = []
        for data in data_loader:
            # Debugging shapes of the incoming data
            # print(f"Batch shape (initial): {data.shape}")

            # Reshape data to the expected dimensions
            data = data.unsqueeze(1)
            feature = data[:, :, 0:-1].to(self.device)
            label = data[:, -1, -1].to(self.device)

            # print(f"Feature shape after reshaping: {feature.shape}")
            # print(f"Label shape after reshaping: {label.shape}")

            # Check for NaNs in the feature and label tensors
            # if torch.isnan(feature).any() or torch.isnan(label).any():
            #     print("NaNs found in feature or label before model prediction.")
            #     print(f"Feature NaN indices: {torch.nonzero(torch.isnan(feature))}")
            #     print(f"Label NaN indices: {torch.nonzero(torch.isnan(label))}")

            pred = self.model(feature.float())

            # Debugging prediction shape
            # print(f"Prediction shape: {pred.shape}")

            # Check for NaNs in prediction
            # if torch.isnan(pred).any():
                # print("NaNs found in predictions before imputation.")
                # print(f"Prediction NaN indices: {torch.nonzero(torch.isnan(pred))}")

            # Impute NaNs after each batch
            feature = self.impute_data(feature, strategy='forward_fill')
            label = self.impute_data(label, strategy='forward_fill')
            pred = self.impute_data(pred, strategy='forward_fill')

            # Check if imputation was successful
            if torch.isnan(feature).any() or torch.isnan(label).any() or torch.isnan(pred).any():
                print("NaNs remain after imputation.")
            else:
                print("NaN handling successful after imputation.")
        return float(np.mean(losses))

    # ==============================================================================
    #
    # REMASTER : ensure slicing operation gets the right inputs
    #
    # ==============================================================================

    def test_epoch(self, data_loader):
        print("========================================================================================================")
        print("Testing...")
        print("========================================================================================================")
        self.set_training_mode(False)  # Evaluation mode
        self.model.eval()
        losses = []

        for data in data_loader:
            # print(f"Batch shape (initial): {data.shape}")

            # Reshape data to expected dimensions
            data = torch.squeeze(data, dim=0)
            feature = data[:, 0:-1].to(self.device)  # Select all columns except the last
            label = data[:, -1].to(self.device)      # Select the last column

            # Adjust feature dimensions to match model expectations
            # Ensure feature's last dimension matches the expected input size of the model
            if feature.shape[-1] > 22:
                feature = feature[:, :, :22]  # Slice to match expected dimension
            elif feature.shape[-1] < 22:
                padding = 22 - feature.shape[-1]
                feature = F.pad(feature, (0, padding), "constant", 0)

            # print(f"Feature shape after adjustment: {feature.shape}")
            # print(f"Label shape: {label.shape}")

            # if torch.isnan(feature).any() or torch.isnan(label).any():
            #     print("NaNs found in feature or label before model prediction.")
            #     print(f"Feature NaN indices: {torch.nonzero(torch.isnan(feature))}")
            #     print(f"Label NaN indices: {torch.nonzero(torch.isnan(label))}")

            # Forward pass
            pred = self.model(feature.float())
            # print(f"Prediction shape: {pred.shape}")

            # Calculate loss
            loss = self.loss_fn(pred, label)
            losses.append(loss.item())

        return float(np.mean(losses))

    # shuffle was og true in MASTER, set false for ReMASTER
    def _init_data_loader(self, data, shuffle=False, drop_last=True):
        sampler = DailyBatchSamplerRandom(data, shuffle)
        data_loader = DataLoader(data, sampler=sampler, drop_last=drop_last)
        # print("DataLoader output:")
        for sampler_idx, sampler in enumerate(data_loader):
            # print(f"Sampler {sampler_idx} - Type: {type(sampler)}, Dtype: {sampler.dtype}")
            # print(f"Sampler shape: {sampler.shape}")
            # print("Sampler contents (first item):", sampler[0])
            break  # Only checking the first batch for debugging
        return data_loader

    def load_param(self, param_path):
        checkpoint = torch.load(param_path, map_location=self.device)
        model_state_dict = self.model.state_dict()

        # Identify mismatched keys
        mismatched_keys = []
        for key, param in checkpoint.items():
            if key in model_state_dict:
                # Check if the shapes match
                if param.shape != model_state_dict[key].shape:
                    # print(f"Skipping loading parameter '{key}' due to shape mismatch: checkpoint shape {param.shape}, model shape {model_state_dict[key].shape}")
                    mismatched_keys.append(key)

        # Remove mismatched keys from checkpoint
        for key in mismatched_keys:
            del checkpoint[key]

        # Load the filtered checkpoint
        self.model.load_state_dict(checkpoint, strict=False)
        # print(f"Model loaded from {param_path} with mismatched layers skipped: {mismatched_keys}")


    def fit(self, dl_train, dl_valid):
        # shuffle was og true in MASTER, set false for ReMASTER
        train_loader = self._init_data_loader(dl_train, shuffle=False, drop_last=True)
        valid_loader = self._init_data_loader(dl_valid, shuffle=False, drop_last=True)

        self.fitted = True
        best_param = None
        for step in range(self.n_epochs):
            train_loss = self.train_epoch(train_loader)
            val_loss = self.test_epoch(valid_loader)

            # ==================================================================================================================================================================
            #
            # remaster none types
            #
            # ==================================================================================================================================================================
            print("Epoch %d, train_loss %.6f, valid_loss %.6f " % (
                step, train_loss if train_loss is not None else -1,
                val_loss if val_loss is not None else -1
            ))
            # print("Epoch %d, train_loss %.6f, valid_loss %.6f " % (step, train_loss, val_loss))
            best_param = copy.deepcopy(self.model.state_dict())

            if train_loss <= self.train_stop_loss_thred:
                break
        torch.save(best_param, f'{self.save_path}{self.save_prefix}master_{self.seed}.pkl')

    # ==================================================================================================================================================================
    #
    # remaster to handle NaN and dimensions
    #
    # ==================================================================================================================================================================

    def predict(self, dl_test):
        if not self.fitted:
            raise ValueError("Model is not fitted yet!")

        test_loader = self._init_data_loader(dl_test, shuffle=False, drop_last=False)
        preds = []
        ic = []
        ric = []

        self.model.eval()
        for idx, data in enumerate(test_loader):
            # print(f"\nBatch {idx} - Initial data shape: {data.shape}")

            # Adjust tensor dimensions if needed
            if data.dim() == 3 and data.shape[0] == 1:
                data = torch.squeeze(data, dim=0)
                # print(f"  Squeezed data shape: {data.shape}")

            if data.dim() == 2:
                feature = data[:, 0:-1].to(self.device)
                label = data[:, -1]
                # print(f"  Feature shape: {feature.shape}, Label shape: {label.shape}")
            else:
                raise ValueError(f"Unexpected tensor shape in DataLoader: {data.shape}")

            with torch.no_grad():
                pred = self.model(feature.float()).detach().cpu().numpy()
                # print(f"  Prediction shape before adjustment: {pred.shape}")

                # Convert pred to tensor if in NumPy format for padding
                if isinstance(pred, np.ndarray):
                    pred = torch.tensor(pred, device=label.device)

                # Adjust pred shape to match label if needed
                if pred.shape[0] < label.shape[0]:
                    padding_size = label.shape[0] - pred.shape[0]
                    pred = F.pad(pred, (0, padding_size), 'constant', 0)
                    # print(f"  Prediction shape after padding: {pred.shape}")
                elif pred.shape[0] > label.shape[0]:
                    pred = pred[:label.shape[0]]
                    # print(f"  Prediction shape after truncating: {pred.shape}")

            # Flatten pred to 1D if necessary
            pred = pred.view(-1)
            label = label.view(-1)

            # Final check to ensure pred and label are the same length
            if pred.shape[0] != label.shape[0]:
                # print(f"WARNING: Prediction and label lengths differ before calc_ic: pred {pred.shape}, label {label.shape}")
                pred = pred[:label.shape[0]]  # Truncate pred to match label length if necessary

            # print(f"Prediction shape going into calc_ic: {pred.shape}, Label shape: {label.shape}")
            preds.append(pred.cpu().numpy())

            # Calculate daily IC and RIC
            daily_ic, daily_ric = calc_ic(pred, label.detach().cpu().numpy())
            if not np.isnan(daily_ic):
                ic.append(daily_ic)
            if not np.isnan(daily_ric):
                ric.append(daily_ric)

        # Concatenate predictions
        try:
            index = dl_test.index.get_level_values('datetime')
        except AttributeError:
            index = range(len(np.concatenate(preds)))
        predictions = pd.Series(np.concatenate(preds), index=index)

        # Final metrics calculation with NaN handling
        mean_ic = np.nanmean(ic)
        std_ic = np.nanstd(ic)
        mean_ric = np.nanmean(ric)
        std_ric = np.nanstd(ric)

        metrics = {
            'IC': mean_ic,
            'ICIR': mean_ic / std_ic if std_ic > 0 else np.nan,
            'RIC': mean_ric,
            'RICIR': mean_ric / std_ric if std_ric > 0 else np.nan
        }

        return predictions, metrics

#--------------------------------------------------------------------------------------------------------------------------------------------
# BACK TO MASTER.PY
#--------------------------------------------------------------------------------------------------------------------------------------------

class MASTERModel(SequenceModel):
    def __init__(
            self, d_feat: int = 20, d_model: int = 64, t_nhead: int = 4, s_nhead: int = 2, gate_input_start_index=None, gate_input_end_index=None,
            T_dropout_rate=0.5, S_dropout_rate=0.5, beta=5.0, **kwargs,
    ):
        super(MASTERModel, self).__init__(**kwargs)
        self.d_model = d_model
        self.d_feat = d_feat

        self.gate_input_start_index = gate_input_start_index
        self.gate_input_end_index = gate_input_end_index

        self.T_dropout_rate = T_dropout_rate
        self.S_dropout_rate = S_dropout_rate
        self.t_nhead = t_nhead
        self.s_nhead = s_nhead
        self.beta = beta

        self.init_model()

    def init_model(self):
        self.model = MASTER(d_feat=self.d_feat, d_model=self.d_model, t_nhead=self.t_nhead, s_nhead=self.s_nhead,
                                   T_dropout_rate=self.T_dropout_rate, S_dropout_rate=self.S_dropout_rate,
                                   gate_input_start_index=self.gate_input_start_index,
                                   gate_input_end_index=self.gate_input_end_index, beta=self.beta)
        super(MASTERModel, self).init_model()

        for param in self.model.parameters():
            param.requires_grad = True

    # ==================================================================================================================================================================
    #
    # remaster to handle NaN and dimensions
    #
    # ==================================================================================================================================================================

    def impute_data(self, tensor, strategy='forward_fill'):
        """
        Impute NaN values in the tensor based on the selected strategy.
        Supported strategies: 'mean', 'median', 'mode', 'forward_fill', 'backward_fill'
        """
        tensor_np = tensor.detach().cpu().numpy()  # Convert to NumPy for easier manipulation

            # Debugging the initial tensor shape and NaN indices
        # print(f"Tensor shape in impute_data: {tensor_np.shape}")
        nan_indices = np.where(np.isnan(tensor_np))
        # print(f"NaN indices: {nan_indices}")

        if strategy == 'mean':
            mean_values = np.nanmean(tensor_np, axis=0)
            # print(f"Mean values shape: {mean_values.shape}")

            # Check if `mean_values` can be indexed correctly
            if len(nan_indices[1]) > 0:
                # print("Applying mean imputation.")
                tensor_np[nan_indices] = np.take(mean_values, nan_indices[1])
            else:
                print("No NaNs found for mean imputation.")
        elif strategy == 'median':
            median_values = np.nanmedian(tensor_np, axis=0)
            indices = np.where(np.isnan(tensor_np))
            if indices[0].size > 0:
                tensor_np[indices] = np.take(median_values, indices[1])

        elif strategy == 'mode':
          from scipy.stats import mode
          mode_values = mode(tensor_np, nan_policy='omit')[0]
          indices = np.where(np.isnan(tensor_np))
          if indices[0].size > 0:
              tensor_np[indices] = np.take(mode_values, indices[1])

        elif strategy == 'forward_fill':
          # Reshape tensor to 2D for forward fill and back to original shape afterward
          # print("Applying forward fill imputation.")
          original_shape = tensor_np.shape
          reshaped_tensor = tensor_np.reshape(-1, original_shape[-1])  # Flatten all but the last dimension
          reshaped_tensor = pd.DataFrame(reshaped_tensor).fillna(method='ffill').to_numpy()
          tensor_np = reshaped_tensor.reshape(original_shape)  # Reshape back to original

        elif strategy == 'backward_fill':
          original_shape = tensor_np.shape
          reshaped_tensor = tensor_np.reshape(-1, original_shape[-1])  # Flatten all but the last dimension
          reshaped_tensor = pd.DataFrame(reshaped_tensor).fillna(method='bfill').to_numpy()
          tensor_np = reshaped_tensor.reshape(original_shape)  # Reshape back to original

        # Convert back to torch tensor and move to the original device
        return torch.tensor(tensor_np, device=tensor.device)



---



## **MAIN.PY**
### **Model Initialization and Training**

In [32]:
import pickle

# ===============================================================================================================================================
#
# ReMASTER: change indexing method with new class
# since the torch.utils.data library DataLoader expects tensors, convert the NQ100 pandas
# df to make the data into tensors that can be indexed
#
# ===============================================================================================================================================
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

class NQ100Dataset(Dataset):
    def __init__(self, dataframe):
        self.data = torch.tensor(dataframe.values, dtype=torch.float32)
        self.index = dataframe.index  # Store the original DataFrame index

        # Print statement to check the tensor type and shape
        print("NQ100Dataset initialized:")
        # print(f"Data type: {type(self.data)}, Tensor dtype: {self.data.dtype}")
        # print(f"Data shape: {self.data.shape}\n")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Print statement for each access to check individual items
        item = self.data[idx]
        # print(f"Accessing index {idx}:")
        # print(f"Item type: {type(item)}, Tensor dtype: {item.dtype}")
        # print(f"Item shape: {item.shape}\n")
        return item


universe = 'NQ100' # or 'csi800'

# Please install qlib first before load the data.
with open(f'/content/drive/My Drive/ECE 570/data/{universe}/{universe}_dl_train.pkl', 'rb') as f:
    dl_train = pickle.load(f)
with open(f'/content/drive/My Drive/ECE 570/data/{universe}/{universe}_dl_valid.pkl', 'rb') as f:
    dl_valid = pickle.load(f)
with open(f'/content/drive/My Drive/ECE 570/data/{universe}/{universe}_dl_test.pkl', 'rb') as f:
    dl_test = pickle.load(f)
print("Data Loaded.")

# ReMASTER tranform to tensors
dl_train = NQ100Dataset(dl_train)
dl_valid = NQ100Dataset(dl_valid)
dl_test = NQ100Dataset(dl_test)

d_feat = 22
d_model = 256
t_nhead = 4
s_nhead = 2
dropout = 0.5
gate_input_start_index=0
gate_input_end_index = 22

if universe == 'csi300':
    beta = 10
elif universe == 'csi800':
    beta = 5
elif universe == 'NQ100':
    beta = 15

n_epoch = 5
lr = 8e-6
GPU = 0
seed = 0
train_stop_loss_thred = 0.95

model = MASTERModel(
    d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
    beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
    n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
    save_path='/content/drive/My Drive/ECE 570/model/', save_prefix=universe
)

# Train
print("***********************************************\n\n\n\n")
print(f"Model Training... for {universe}")
print("***********************************************\n\n\n\n")
model.fit(dl_train, dl_valid)
print("Model Trained.")

# Test
print("***********************************************\n\n\n\n")
print(f"Model Testing... for {universe}\n\n\n\n")
print("***********************************************\n\n\n\n")
predictions, metrics = model.predict(dl_test)
print("***********************************************\n\n\n\n")
print(f"Retrieving Metrics... for {universe}\n\n\n\n")
print("***********************************************\n\n\n\n")
print(metrics)
print("Model Tested.")


Data Loaded.
NQ100Dataset initialized:
NQ100Dataset initialized:
NQ100Dataset initialized:
***********************************************




Model Training... for NQ100
***********************************************




Training...
NaNs remain after imputation.
NaNs remain after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imp

<ipython-input-29-672450df5be9>:50: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.daily_count = pd.Series(index=self.data_source.index.get_level_values('datetime')).groupby("datetime").size().values
<ipython-input-29-672450df5be9>:50: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.daily_count = pd.Series(index=self.data_source.index.get_level_values('datetime')).groupby("datetime").size().values


NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after impu

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after imputation.
NaN handling successful after impu

<ipython-input-29-672450df5be9>:50: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.daily_count = pd.Series(index=self.data_source.index.get_level_values('datetime')).groupby("datetime").size().values
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


***********************************************




Retrieving Metrics... for NQ100




***********************************************




{'IC': -0.00019879946773001533, 'ICIR': -0.0024835037657962365, 'RIC': -0.0056678471750652775, 'RICIR': -0.06334532389901873}
Model Tested.


### **Model Loading and Testing**

In [33]:
import os
# Load and Test

# Load the checkpoint
param_path = f'/content/drive/My Drive/ECE 570/model/{universe}master_0.pkl'
checkpoint = torch.load(param_path, map_location="cpu")

# Resize the positional encoding if the key exists
if 'layers.1.pe' in checkpoint:
    pe_tensor = checkpoint['layers.1.pe']

    # Determine the target shape
    target_shape = (276, 256)

    if pe_tensor.shape[0] < target_shape[0]:
        # Pad rows if current height is less than target height
        padding = (0, 0, 0, target_shape[0] - pe_tensor.shape[0])
        resized_pe = torch.nn.functional.pad(pe_tensor, padding, mode='constant', value=0)
    else:
        # Truncate rows if current height is greater than target height
        resized_pe = pe_tensor[:target_shape[0], :]

    # Update the checkpoint with the resized `pe`
    checkpoint['layers.1.pe'] = resized_pe

# Save the modified checkpoint back to the file
torch.save(checkpoint, param_path)

# Now load the adjusted checkpoint into the model
model.load_param(param_path)
# param_path = f'/content/drive/My Drive/ECE 570/model/{universe}master_0.pkl'
print(f'Model Loaded from {param_path}')
model.load_param(param_path)
predictions, metrics = model.predict(dl_test)
print("***********************************************\n\n\n\n")
print(f"Retrieving Metrics... for {universe}\n\n\n\n")
print("***********************************************\n\n\n\n")
print(metrics)
print("\n\n\n\n***********************************************\n\n\n\n")
print(f"Goodbye! :)\n\n\n\n")
print("***********************************************\n\n\n\n")

Model Loaded from /content/drive/My Drive/ECE 570/model/NQ100master_0.pkl


<ipython-input-29-672450df5be9>:50: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self.daily_count = pd.Series(index=self.data_source.index.get_level_values('datetime')).groupby("datetime").size().values
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


***********************************************




Retrieving Metrics... for NQ100




***********************************************




{'IC': -0.00019879946773001533, 'ICIR': -0.0024835037657962365, 'RIC': -0.0056678471750652775, 'RICIR': -0.06334532389901873}




***********************************************




Goodbye! :)




***********************************************






# This is where the official ReMASTER ends. Everything below is experimental :) Happy Investing!




---



---



---



---



---

